# Prep

In [1]:
import musicbrainzngs
import pandas as pd

from ratelimit import limits, sleep_and_retry
import traceback

import numpy as np

# Authentication

In [2]:
musicbrainzngs.set_useragent(app="Opus Metadata Fetcher",version = "1.0")

# Add Musicbrainz metadata

## Utility Functions

In [3]:
def mb_recording_template(input_isrc = ''):
    # default recording_id in the format of music brainz recording id
    default_recording_id = 'zzzzzzzz-zzzz-zzzz-zzzz-zzzzzzzzzzzz'
    default_mb_recording_title = ''
    default_mb_recording_length = '0'
    
    data = {
    'mb_recording_id': [default_recording_id],
    'mb_recording_title': [default_mb_recording_title],
    'mb_recording_length': [default_mb_recording_length],
    'isrc_opus': [input_isrc]
}
    default_recording_df = pd.DataFrame(data)
    
    return default_recording_df

def compare_dataframes(df1, df2):
    # Check if column names are equal
    if list(df1.columns) != list(df2.columns):
        print("Column names are not equal")
        return False
    
    # Check if column types are equal
    if not df1.dtypes.equals(df2.dtypes):
        print("Column types are not equal")
        return False
    
    # print("Column names and types are equal")
    return True
def create_mb_globals():
    # dictionary of globals
    
    # default df
    mb_recording_df_dflt = mb_recording_template()
    
    found_recording_dfs = []
    missing_recording_dfs = []
    mb_release_dfs = []
    mb_label_dfs = []
    error_log = []

    mb_global_dict = {'recording_df_default':mb_recording_df_dflt,
                      'found_recording_dfs':found_recording_dfs,
                      'missing_recording_dfs':missing_recording_dfs,
                      'release_dfs':mb_release_dfs,
                      'label_dfs':mb_label_dfs,
                      'error_log':error_log}
    return mb_global_dict




def format_recording(mb_recording,input_isrc):
    # formats music brains recording object
    recording_df = pd.DataFrame(mb_recording['isrc']['recording-list'])
    recording_df['isrc_opus'] = input_isrc
    recording_df = recording_df.rename(columns={'id':'mb_recording_id',
                                                    'title':'mb_recording_title',
                                                    'length':'mb_recording_length'})
    return recording_df

In [4]:
# consolidate dataframes


### Release

In [5]:
def parse_release_text(text):
    """retrieve language from text-representation field of release dataframe
    ex: {'language': 'eng', 'script': 'Latn'} --> 'eng' """
    try: 
        # format string as a dictionary and extract value of 'language' key
        if pd.isna(text):
            language = 'zzz'
        else:
            language = dict(text)['language']
    except TypeError as e:
        language = 'zzz'
        print(e)
    finally:
        return language
def parse_release_date(date_str):
    # handle missing values
    if pd.isna(date_str):
        return '1900-01-01'
    elif len(date_str) == 4:
        return date_str + '-01-01'  # Assuming it's the start of the year
    elif len(date_str) == 7:
        return date_str + '-01'
    else:
        return date_str
    
def flag_complete_date(date_str):
    try:
        pd.to_datetime(date_str,format='%Y-%m-%d')
        return '1'
    except ValueError:
        return '0'
    except TypeError:
        return '0'

def format_mb_release(mb_release,recording_id):
    release_df = pd.DataFrame(mb_release['release-list'])
    # print(mb_release.columns)
    # get release language from text-representation field
    release_df['mb_release_language'] = release_df['text-representation'].apply(parse_release_text)
    # flag date format
    release_df['mb_complete_date_flag'] = release_df['date'].apply(flag_complete_date)
    # format date
    release_df['mb_date'] = release_df['date'].apply(parse_release_date)
    
    # add recording_id
    release_df['mb_recording_id'] = recording_id
    # rename columns
    release_df = release_df.rename(columns = {'id':'mb_release_id',
                                              'title':'mb_release_title',
                                              'status':'mb_release_status',
                                              'country':'mb_release_country',
                                              'quality':'mb_release_quality'})
    
    release_df['recording_id'] = recording_id
    # select columns
    release_df = release_df[['mb_release_id',
                             'mb_release_title',
                             'mb_release_status',
                             'mb_release_country',
                             'mb_release_quality',
                             'mb_release_language',
                             'mb_complete_date_flag',
                             'mb_date',
                             'mb_recording_id']]
    return release_df
    # print('done')


In [6]:
# Decorate the function with sleep_and_retry
@sleep_and_retry
# Set the rate limit to 1 calls per second
@limits(calls=1, period=1)
# define function
def build_release_df(input_recording_id,mb_global_dict):
    print(input_recording_id)
    # if HTTPError, handle with default df
    # try:
        # look up release using recording_id
    print('searching for release')
    releases = musicbrainzngs.browse_releases(recording=input_recording_id)
    print('formatting release')
    release_df = format_mb_release(releases,recording_id=input_recording_id)
    print('adding to global dict')
    mb_global_dict['release_dfs'].append(release_df)
    # except musicbrainzngs.musicbrainz.ResponseError as e:
    #     print('recording not found in musicbrainz')

    # finally:
    #     return mb_global_dict

## Build recording df

In [7]:
# plan
mb_global_dict = create_mb_globals()
# isrc --> recording --> release --> label
input_isrcs = ['USSM10600677',
               'USSM106006771',
               'USSM106006772',
               'USSM106006773',
               'USSM106006774',
               'USSM106006775']
# Decorate the function with sleep_and_retry
@sleep_and_retry
# Set the rate limit to 1 calls per second
@limits(calls=1, period=1)
def build_recording_df(input_isrc,mb_global_dict):
    # if HTTPError, handle with default df
    try:
        # look up recording using isrc. if the isrc is not found, ResponseError will be thrown
        recordings = musicbrainzngs.get_recordings_by_isrc(input_isrc, includes=[], release_status=[], release_type=[])
        # if isrc is found, then format the recording information
        recording_df = format_recording(recordings,input_isrc = input_isrc)
        assert compare_dataframes(recording_df,mb_global_dict['recording_df_default'])
        # add to list of found recordings
        mb_global_dict['found_recording_dfs'].append(recording_df)
    except musicbrainzngs.musicbrainz.ResponseError as e:
        print('isrc not found in musicbrainz')
        # create missing recording info for isrc
        recording_df = mb_recording_template(input_isrc=input_isrc)
        # ensure proper format 
        assert compare_dataframes(recording_df,mb_global_dict['recording_df_default'])
        # add to missing recording dfs
        mb_global_dict['missing_recording_dfs'].append(recording_df)
    
    # except Exception as e:
    #     print(e)
    #     error_log = {'process':'recording_df',
    #                  'error_type':'fatal',
    #                  'error':e}
    #     mb_global_dict['error_log'].append(error_log)
        # raise e
        
    
    finally:
        return mb_global_dict

# dim_song_detail --> stg_mb_recording_detail (key: isrc, mb_recording_id)
# stg_mb_release_detail (key: recording/release)
# stg_mb_label_detail (key: label/release)

In [8]:
# Call the function ensuring one call per second
for isrc in input_isrcs:
    mb_global_dict = build_recording_df(input_isrc=isrc,mb_global_dict = mb_global_dict)


isrc not found in musicbrainz
isrc not found in musicbrainz
isrc not found in musicbrainz
isrc not found in musicbrainz
isrc not found in musicbrainz


## Build release df

In [9]:
for df in mb_global_dict['found_recording_dfs']:
    for recording_id in df['mb_recording_id']:
        build_release_df(recording_id,mb_global_dict)

53056bc7-d60e-4f68-b420-11894097fc09
searching for release
formatting release
adding to global dict
2c0b2787-307c-49e9-b2e9-5b1ddd6e713e
searching for release
formatting release
adding to global dict


In [10]:
mb_global_dict['release_dfs'][1]

,mb_release_id,mb_release_title,mb_release_status,mb_release_country,mb_release_quality,mb_release_language,mb_complete_date_flag,mb_date,mb_recording_id
0,5fe133cb-d394-4089-a5e1-ec1e27bf4fe3,"Oral Fixation, Vol. 2",Official,BR,normal,eng,1,2005-11-28,2c0b2787-307c-49e9-b2e9-5b1ddd6e713e
1,218a7876-c3f1-4a48-bb21-a68fc69bc5f2,"Promo Only: Mainstream Radio, March 2006",Promotion,US,normal,eng,1,2006-02-17,2c0b2787-307c-49e9-b2e9-5b1ddd6e713e
2,7305d111-b4cf-49e1-a8c6-df46c2285706,Hips Don’t Lie,Official,XW,normal,eng,1,2006-03-15,2c0b2787-307c-49e9-b2e9-5b1ddd6e713e
3,78ae2052-d68b-4cb6-9de4-917311094de1,"Oral Fixation, Vol. 2",Official,XW,normal,eng,1,2006-03-27,2c0b2787-307c-49e9-b2e9-5b1ddd6e713e
4,a4fe238c-d6a1-4c12-97c0-a516b1fd7868,"Oral Fixation, Vol. 2",Official,AU,normal,eng,1,2006-03-28,2c0b2787-307c-49e9-b2e9-5b1ddd6e713e
5,4102ba3c-a576-43f5-bc38-49c5b42ef03a,"Oral Fixation, Vol. 2",Official,US,normal,eng,1,2006-03-28,2c0b2787-307c-49e9-b2e9-5b1ddd6e713e
6,b39ed78f-808b-4261-a0d4-348a6168f08a,Hips Don’t Lie,Official,XE,normal,eng,1,2006-04-17,2c0b2787-307c-49e9-b2e9-5b1ddd6e713e
7,529fea5d-60c9-4e54-af3a-87355c09f454,Hips Don't Lie,Official,IN,normal,eng,0,2006-04-01,2c0b2787-307c-49e9-b2e9-5b1ddd6e713e
8,afef6546-2432-4a97-8fe1-21c5ff05d788,Hips Don't Lie,Official,DE,normal,eng,1,2006-05-05,2c0b2787-307c-49e9-b2e9-5b1ddd6e713e
9,76b56517-873f-4a5b-95f1-7dda12b148f2,Hips Don't Lie,Official,DE,normal,eng,1,2006-05-05,2c0b2787-307c-49e9-b2e9-5b1ddd6e713e


# Scratch Work

In [11]:
test = mb_global_dict['release_dfs'][0]
test

,mb_release_id,mb_release_title,mb_release_status,mb_release_country,mb_release_quality,mb_release_language,mb_complete_date_flag,mb_date,mb_recording_id
0,27af023b-7967-4cc4-8ef5-7edccada8219,Die ultimative Chart Show: Die erfolgreichsten...,Official,DE,normal,mul,1,2009-11-13,53056bc7-d60e-4f68-b420-11894097fc09
1,5215b593-5741-47b3-bd4e-d5a3d473068f,School Disco Anthems,Official,AU,normal,eng,1,2010-05-25,53056bc7-d60e-4f68-b420-11894097fc09
2,ee5ac04a-143f-4371-b90f-5c8e0b5e5751,Zomer Top 100: Joe FM,Official,BE,normal,eng,1,2012-06-22,53056bc7-d60e-4f68-b420-11894097fc09


In [12]:
label_info = musicbrainzngs.browse_labels(release='27af023b-7967-4cc4-8ef5-7edccada8219')

In [13]:
def format_mb_label(label_info,release_record):
    # add release/recording id
    mb_release_id = release_record['mb_release_id']
    mb_recording_id = release_record['mb_recording_id']
    
    # format label from API call
    label_df = pd.DataFrame(label_info['label-list'])
    
    # add release/recording info
    label_df['mb_release_id'] = mb_release_id
    label_df['mb_recording_id'] = mb_recording_id
    
    # rename columns
    label_df = label_df.rename(columns = {'id':'mb_label_id',
                                          'name':'mb_label_name',
                                          'country':'mb_label_country',
                                          })
    
    # select columns
    label_df = label_df[['mb_label_id','mb_release_id','mb_recording_id','mb_label_name','mb_label_country']]
    
    return(label_df)

format_mb_label(label_info, test.iloc[0])

,mb_label_id,mb_release_id,mb_recording_id,mb_label_name,mb_label_country
0,7fdf4b1f-37fb-4d5e-9445-b17b0a6930e4,27af023b-7967-4cc4-8ef5-7edccada8219,53056bc7-d60e-4f68-b420-11894097fc09,Polystar,DE


In [14]:
# Decorate the function with sleep_and_retry
@sleep_and_retry
# Set the rate limit to 1 calls per second
@limits(calls=1, period=1)
# define function
def build_label_df(release_record,mb_global_dict):
    # input: row from release_df, mb_global_dict
    # output: record for label_df added to global_dict
    release_id = release_record['mb_release_id']
    # if HTTPError, handle with default df
    try:
        # look up recording using isrc. if the isrc is not found, ResponseError will be thrown
        label_info = musicbrainzngs.browse_labels(release=release_id)
        label_df = format_mb_label(label_info,release_record)
        mb_global_dict['label_dfs'].append(label_df)
        # mb_global_dict['release_dfs'].append(release_df)
    except musicbrainzngs.musicbrainz.ResponseError as e:
        print('label not found in musicbrainz')

    finally:
        return mb_global_dict

In [15]:
release_record = mb_global_dict['release_dfs'][0].iloc[0]
release_record

build_label_df(release_record,mb_global_dict)

{'recording_df_default':                         mb_recording_id mb_recording_title  \
 0  zzzzzzzz-zzzz-zzzz-zzzz-zzzzzzzzzzzz                      
 
   mb_recording_length isrc_opus  
 0                   0            ,
 'found_recording_dfs': [                        mb_recording_id              mb_recording_title  \
  0  53056bc7-d60e-4f68-b420-11894097fc09  Hips Don’t Lie (clean version)   
  1  2c0b2787-307c-49e9-b2e9-5b1ddd6e713e                  Hips Don’t Lie   
  
    mb_recording_length     isrc_opus  
  0              221000  USSM10600677  
  1              218186  USSM10600677  ],
 'missing_recording_dfs': [                        mb_recording_id mb_recording_title  \
  0  zzzzzzzz-zzzz-zzzz-zzzz-zzzzzzzzzzzz                      
  
    mb_recording_length      isrc_opus  
  0                   0  USSM106006771  ,
                          mb_recording_id mb_recording_title  \
  0  zzzzzzzz-zzzz-zzzz-zzzz-zzzzzzzzzzzz                      
  
    mb_recording_length     

In [16]:
mb_global_dict.keys()

dict_keys(['recording_df_default', 'found_recording_dfs', 'missing_recording_dfs', 'release_dfs', 'label_dfs', 'error_log'])

## end to end

In [17]:
# plan
# initialize global dictionary to store dataframes
mb_global_dict = create_mb_globals()
# isrc --> recording --> release --> label
input_isrcs = ['US23S9528103',
               'USPO19000001']
# 
# create recording dfs
for isrc in input_isrcs:
    mb_global_dict = build_recording_df(input_isrc=isrc,mb_global_dict = mb_global_dict)

mb_global_dict['recording_df'] = pd.concat(mb_global_dict['found_recording_dfs'])

mb_global_dict['recording_df']


,mb_recording_id,mb_recording_title,mb_recording_length,isrc_opus
0,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,Eternal,243893,US23S9528103
0,828debce-b03a-4560-bb71-8454494c3c41,100 Miles and Runnin’,272933,USPO19000001


In [18]:
for recording_id in mb_global_dict['recording_df']['mb_recording_id']:
    # print(recording_id)
    build_release_df(recording_id,mb_global_dict)

01f2ed7c-d42d-4612-af27-1f3dfaeceed3
searching for release
formatting release
adding to global dict
828debce-b03a-4560-bb71-8454494c3c41
searching for release
formatting release
adding to global dict


In [20]:
mb_global_dict['release_df'] = pd.concat(mb_global_dict['release_dfs'])

In [21]:
for idx,row in mb_global_dict['release_df'].iterrows():
    build_label_df(row,mb_global_dict)

In [22]:
mb_global_dict['label_df'] = pd.concat(mb_global_dict['label_dfs'])

In [23]:
mb_global_dict['label_df']

,mb_label_id,mb_release_id,mb_recording_id,mb_label_name,mb_label_country
0,52fa5738-5b7c-4748-b239-3b867504c46a,67d40d9f-2c89-4d49-8401-794e34f3e2c4,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,Ruthless Records,US
0,52fa5738-5b7c-4748-b239-3b867504c46a,a4579d95-44bc-3de4-8782-57b3872c4bcf,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,Ruthless Records,US
0,52fa5738-5b7c-4748-b239-3b867504c46a,cd4d69ae-98a0-4e5b-b512-722f71254a58,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,Ruthless Records,US
0,52fa5738-5b7c-4748-b239-3b867504c46a,98b5846e-4041-39a7-b200-846f7516fafd,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,Ruthless Records,US
0,52fa5738-5b7c-4748-b239-3b867504c46a,5e1806ad-6f0c-4894-b88f-486ee5717afb,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,Ruthless Records,US
0,52fa5738-5b7c-4748-b239-3b867504c46a,b71bfa10-25b0-457a-8b21-54e5e8e822ea,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,Ruthless Records,US
0,52fa5738-5b7c-4748-b239-3b867504c46a,5852bacd-8636-4683-9182-16fc19eebf5d,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,Ruthless Records,US
0,52fa5738-5b7c-4748-b239-3b867504c46a,099443c7-2578-46e7-8729-25b66aeeeae0,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,Ruthless Records,US
1,66aa41b9-744c-41c0-b925-ffe3fee9b236,099443c7-2578-46e7-8729-25b66aeeeae0,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,"Vinyl Me, Please.",US
0,52fa5738-5b7c-4748-b239-3b867504c46a,d89f94fa-9a2b-4eb9-ae2a-53e0806eec55,01f2ed7c-d42d-4612-af27-1f3dfaeceed3,Ruthless Records,US


In [ ]:
mb_global_dict.keys()

dict_keys(['recording_df_default', 'found_recording_dfs', 'missing_recording_dfs', 'release_dfs', 'label_dfs', 'error_log', 'recording_df'])